In [ ]:
# code sketch:
# 1. small subset of songs, get lyrics / info from spotify / genius api
#     turn lyrics coupled w metadata into vector using BERT or something
#.    store in Pinecone? FAISS
# 2. user input > gpt 4 / langchain to get semantic vector rep. of that
#.     compare using cosine sim., closest 10 get suggested


In [ ]:
# SPOTIFY / GENIUS END:
#.   (first, parse user input, find vector vs. "energy" and other things that could narrow the search)
#    pick 100 random songs that work > get their metadata and send to genius > extract lyrics, save as text files
#          if no lyrics, reject

In [ ]:
# LANGCHAIN:
#   turn the lyrics into vectors > store in pinecone
#   user input from gpt into langchain > comapre to vector store
#   Find top 10 songs in song names > send back to spotify to pull up those songs as playable

In [ ]:
# Global imports
from tqdm import tnrange, tqdm_notebook, tqdm
from time import sleep
from lyricsgenius import Genius
import time
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import random

from langchain.document_loaders import PyPDFLoader, TextLoader
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter, CharacterTextSplitter
from langchain.vectorstores import DeepLake
from langchain.chains import ConversationalRetrievalChain, RetrievalQA
from langchain.chat_models import ChatOpenAI
from langchain.llms import OpenAI
from langchain.vectorstores import FAISS
from langchain.vectorstores import Pinecone


#get my secret sauce variables
from dotenv import load_dotenv
import os
load_dotenv() 

PINECONE_API_KEY = os.getenv('PINECONE_API_KEY')
openai_api_key = os.getenv('OPENAI_API_KEY')
SPOTIFY_CLIENT_ID = os.getenv('SPOTIFY_CLIENT_ID')
SPOTIFY_CLIENT_SECRET=os.getenv('SPOTIFY_CLIENT_SECRET')
token = os.getenv('GENIUS_ACCESS_TOKEN')

In [181]:
#Spotify Handling, where we get the 100 sorted, random songs with lyrics:

# need to add song length detection, nothing over 6 mins.

#Authentication - without user
client_credentials_manager = SpotifyClientCredentials(client_id='36c36f81c0ff4b9da4cdbb44d12de650', client_secret='4ecf40e8b9f5443bae664b740a319465')
sp = spotipy.Spotify(client_credentials_manager = client_credentials_manager)

#pick 100 random songs

import random
import string

#this part basically picks random characters from the list of all characters

def get_random_search():
    # A list of all characters that can be chosen.
    characters = string.ascii_lowercase

    # Gets a random character from the characters string.
    random_character = random.choice(characters)
    random_search = ''

    # Places the wildcard character at the beginning, or both beginning and end, randomly.
    switch = random.randint(0, 1)
    if switch == 0:
        random_search = random_character + '%'
    elif switch == 1:
        random_search = '%' + random_character + '%'

    return random_search


trackList = [] #empty tracklist array we're gonna fill

songNumber = 10000 # number of songs we're gonna sample from randomly
pbar = tqdm(total = songNumber) #progress bar init

while len(trackList) <= songNumber: 
    try:
        ranNum = random.randint(1,600) # make sure the offset is a random one so there's never repeats, and they aren't all super pop. new.
        query = get_random_search() #use the function to get a random search on a letter, ie. e% so any song that starts with that letter
        track = sp.search(query, limit=1, offset = ranNum, type='track') #query spotify
        trackURI = track['tracks']['items'][0]['uri']
        trackURI = trackURI.replace('spotify:album:', '')
        trackName = track['tracks']['items'][0]['name']
        trackArtist = track['tracks']['items'][0]['artists'][0]['name']
        audio_features = sp.audio_features(trackURI)
        
        # Check if audio_features is not None and is not an empty list
        if audio_features and len(audio_features) > 0:
            instrumentalness = audio_features[0]['instrumentalness'] #does this have lyrics?
        else:
            continue

        if instrumentalness <= 0.5: #if no lyrics, reject and get a new one
            sleep(0.1)
            pbar.update(1)
            array = [trackName, trackArtist]
            trackList.append(array)
    except Exception as e:
        print(f"Error: {e}")
        continue




  2%|▊                                    | 218/10000 [01:53<1:30:06,  1.81it/s]

Error: list index out of range


  2%|▊                                    | 223/10000 [01:56<1:30:29,  1.80it/s]

Error: 'NoneType' object is not subscriptable


 25%|████████▊                           | 2454/10000 [22:16<1:05:07,  1.93it/s]

...

 75%|███████████████████████████         | 7522/10000 [1:08:04<21:48,  1.89it/s]

Error: list index out of range


 89%|███████████████████████████████▉    | 8881/10000 [1:20:13<10:11,  1.83it/s]

Error: 'NoneType' object is not subscriptable


 91%|████████████████████████████████▋   | 9075/10000 [1:21:54<07:01,  2.19it/s]IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [ ]:
while len(trackList) <= songNumber: 
    ranNum = random.randint(1,600) # make sure the offset is a random one so there's never repeats, and they aren't all super pop. new.
    query = get_random_search() #use the function to get a random search on a letter, ie. e% so any song that starts with that letter
    track = sp.search(query, limit=1, offset = ranNum, type='track') #query spotify
    trackURI = track['tracks']['items'][0]['uri']
    trackURI = trackURI.replace('spotify:album:', '')
    trackName = track['tracks']['items'][0]['name']
    trackArtist = track['tracks']['items'][0]['artists'][0]['name']
    audio_features = sp.audio_features(trackURI)
    instrumentalness = audio_features[0]['instrumentalness'] #does this have lyrics?
    
    if instrumentalness <= 0.5: #if no lyrics, reject and get a new one
        sleep(0.1)
        pbar.update(1)
        array = [trackName, trackArtist]
        trackList.append(array)

        track = None


In [182]:
# GENIUS PART , now we take the song lists and download the lyrics for each one into separate text files for parsing with LangCHAIN:
# going to use lyricsgenius which is a python wrapper for GENIUS API
genius = Genius(token)

import time
import lyricsgenius
from requests.exceptions import Timeout

genius = lyricsgenius.Genius("my_token")  # replace with your token
genius.verbose = False

#custom get song lyrics function, with timeout behavior
def get_song_lyrics(title, artist, max_retries=5, delay_between_retries=10):
    for attempt in range(max_retries):
        try:
            song = genius.search_song(title=title, artist=artist)
            if song is not None:
                return song
            else:
                print(f"No lyrics found for {title} by {artist}.")
                return None
        except Timeout:
            if attempt < max_retries - 1:  # it's not the last attempt
                print(f"Timeout occurred when getting lyrics for {title} by {artist}. "
                      f"Waiting {delay_between_retries} seconds before retrying...")
                time.sleep(delay_between_retries)
                continue
            else:  # it's the last attempt
                print(f"Still experiencing timeouts after {max_retries} attempts. "
                      f"Giving up on getting lyrics for {title} by {artist}.")
                return None
        except Exception as e:
            print(f"An error occurred: {e}")
            return None

def sanitize_filename(name):
    for char in ['/', '\\', ':', '*', '?', '"', '<', '>', '|']:
        name = name.replace(char, '_')
    return name

for i in tqdm(trackList):
    title = sanitize_filename(i[0])
    artist = sanitize_filename(i[1])
    song = get_song_lyrics(title, artist,5,10)
    if song != None and "[Verse" in song.lyrics:
        lyrics = song.lyrics
        with open('lyrics/%s_%s.txt'%(title,artist), 'w') as f:
            f.write(lyrics)
            


  0%|                                      | 1/10001 [00:03<10:48:25,  3.89s/it]

Timeout occurred when getting lyrics for We Built This City by Starship. Waiting 10 seconds before retrying...



  0%|                                       | 6/10001 [00:28<8:30:25,  3.06s/it]

No lyrics found for novocaine by GenriX.



  0%|                                      | 30/10001 [01:16<5:17:19,  1.91s/it]

...


100%|███████████████████████████████████▉| 9975/10001 [6:13:42<00:27,  1.06s/it]

Timeout occurred when getting lyrics for Stay With Me by Sam Smith. Waiting 10 seconds before retrying...



100%|███████████████████████████████████▉| 9977/10001 [6:13:59<01:36,  4.02s/it]

No lyrics found for Mo Money Mo Problems (feat. Puff Daddy & Mase) - 2014 Remaster by The Notorious B.I.G..



100%|███████████████████████████████████| 10001/10001 [6:14:21<00:00,  2.25s/it]


In [202]:
#LANGCHAIN TIME
# here gonna take each lyric file and summarize it / vectorize it and store in pinecone
# initialize langchain / openAI
import pinecone
from langchain.chains.summarize import load_summarize_chain
embeddings = OpenAIEmbeddings(openai_api_key = openai_api_key)
llm = OpenAI(temperature=0)
chain = load_summarize_chain(llm, chain_type="stuff")

docs=[]
summary = []

for filename in tqdm(os.listdir(directory)):
    if filename.endswith(".txt"):  # check if the file is a .txt file
        file_path = os.path.join(directory, filename)
        # now you can do something with the file
        loader = TextLoader(file_path)
        doc = loader.load()
        docs.append(doc)
for i in tqdm(docs):
    summary.append(chain.run(i))

        


100%|█████████████████████████████████████| 2766/2766 [00:00<00:00, 6026.63it/s]

  3%|█▎                                     | 91/2765 [09:31<4:18:42,  5.81s/it]


  7%|██▌                                   | 188/2765 [19:45<4:49:46,  6.75s/it]


 10%|███▉                                  | 285/2765 [30:29<4:13:27,  6.13s/it]


 14%|█████▏                                | 382/2765 [40:49<4:06:51,  6.22s/it]


 17%|██████▌                               | 479/2765 [49:50<3:15:21,  5.13s/it]


...

 24%|████████▊                           | 673/2765 [1:10:18<3:22:15,  5.80s/it]


 28%|█████████▉                          | 763/2765 [1:18:37<2:14:56,  4.04s/it]Retrying langchain.llms.openai.completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised RateLimitError: The server had an error while processing your request. Sorry about that!.

 28%|█████████▉                          | 768/2765 [1:19:02<2:32:09,  4.57s/it]


 31%|███████████▎                        | 865/2765 [1:30:09<4:19:41,  8.20s/it]


 34%|████████████▍                       | 952/2765 [1:39:27<3:09:23,  6.27s/it]


InvalidRequestError: This model's maximum context length is 4097 tokens, however you requested 5571 tokens (5315 in your prompt; 256 for the completion). Please reduce your prompt; or completion length.

In [184]:
pinecone.init(
    api_key=PINECONE_API_KEY,
    environment='asia-northeast1-gcp'
)

index_name="spotifysemantic"

for i in tqdm(docs):
    docsearch=Pinecone.from_documents(i, embeddings, index_name=index_name)






  3%|█▎                                     | 96/2765 [05:06<2:57:18,  3.99s/it]


  7%|██▋                                   | 193/2765 [10:35<2:17:05,  3.20s/it]


 10%|███▉                                  | 290/2765 [15:46<1:56:02,  2.81s/it]


 14%|█████▎                                | 387/2765 [20:52<1:57:14,  2.96s/it]


 18%|██████▋                               | 484/2765 [25:39<1:53:52,  3.00s/it]


...

 87%|████████████████████████████████▏    | 2403/2765 [2:10:29<17:22,  2.88s/it]


 90%|█████████████████████████████████▍   | 2500/2765 [2:15:14<12:19,  2.79s/it]


 94%|██████████████████████████████████▊  | 2597/2765 [2:19:55<07:44,  2.76s/it]


 97%|████████████████████████████████████ | 2694/2765 [2:24:42<03:58,  3.35s/it]


100%|█████████████████████████████████████| 2765/2765 [2:28:12<00:00,  3.22s/it]


In [194]:
docsearch = Pinecone.from_existing_index(index_name, embeddings)
query = "songs for the best day of my life"
ans = docsearch.similarity_search(query)

for i in ans:
    print(i.metadata)

{'source': 'lyrics/Best Day Of My Life_American Authors.txt'}
{'source': 'lyrics/The Greatest Day of My Life_Zach Bryan.txt'}
{'source': 'lyrics/Joy Of My Life_Chris Stapleton.txt'}
{'source': "lyrics/It's A Great Day To Be Alive_Travis Tritt.txt"}
